### Global Sequence Alignment (Needleman-Wunsch Algorithm)


In [13]:
def needleman_wunsch(seq1, seq2, match_score=1, mismatch_score=-1, gap_penalty=-1):
    n = len(seq1)
    m = len(seq2)

    # Initialize the score matrix
    score_matrix = [[0] * (m + 1) for _ in range(n + 1)]

    # Initialize the traceback matrix to store pointers for alignment reconstruction
    traceback_matrix = [[0] * (m + 1) for _ in range(n + 1)]

    # Initialize the first row and column of the score matrix and traceback matrix
    for i in range(1, n + 1):
        score_matrix[i][0] = score_matrix[i-1][0] + gap_penalty
        traceback_matrix[i][0] = 1 # Pointer from above
    for j in range(1, m + 1):
        score_matrix[0][j] = score_matrix[0][j-1] + gap_penalty
        traceback_matrix[0][j] = 2 # Pointer from left

    # Fill the score matrix and traceback matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match = score_matrix[i-1][j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score)
            delete = score_matrix[i-1][j] + gap_penalty  # Gap in seq2
            insert = score_matrix[i][j-1] + gap_penalty  # Gap in seq1

            score_matrix[i][j] = max(match, delete, insert)

            if score_matrix[i][j] == match:
                traceback_matrix[i][j] = 0 # Diagonal
            elif score_matrix[i][j] == delete:
                traceback_matrix[i][j] = 1 # Up
            else:
                traceback_matrix[i][j] = 2 # Left

    # Print the score matrix with sequence labels
    print("\nNeedleman-Wunsch Score Matrix:")
    header_elements = ["   "] + [" _ "] + [f"{char:3s}" for char in seq2]
    print("".join(header_elements))

    for i in range(n + 1):
        row_prefix = [" _ "] if i == 0 else [f"{seq1[i-1]:3s}"]
        row_values = [f"{val:3d}" for val in score_matrix[i]]
        print("".join(row_prefix + row_values))

    # Traceback to reconstruct the alignment
    aligned_seq1 = []
    aligned_seq2 = []
    i, j = n, m

    while i > 0 or j > 0:
        if traceback_matrix[i][j] == 0: # Diagonal
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append(seq2[j-1])
            i -= 1
            j -= 1
        elif traceback_matrix[i][j] == 1: # Up (gap in seq2)
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append('-')
            i -= 1
        else: # Left (gap in seq1)
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j-1])
            j -= 1

    return ''.join(aligned_seq1[::-1]), ''.join(aligned_seq2[::-1]), score_matrix[n][m]

# Example Usage for Needleman-Wunsch
seq1_nw = "GAATTC"
seq2_nw = "GATTAAC"
aligned1_nw, aligned2_nw, score_nw = needleman_wunsch(seq1_nw, seq2_nw)

print(f"Needleman-Wunsch Global Alignment:")
print(f"Sequence 1: {aligned1_nw}")
print(f"Sequence 2: {aligned2_nw}")
print(f"Alignment Score: {score_nw}")


Needleman-Wunsch Score Matrix:
    _ G  A  T  T  A  A  C  
 _   0 -1 -2 -3 -4 -5 -6 -7
G   -1  1  0 -1 -2 -3 -4 -5
A   -2  0  2  1  0 -1 -2 -3
A   -3 -1  1  1  0  1  0 -1
T   -4 -2  0  2  2  1  0 -1
T   -5 -3 -1  1  3  2  1  0
C   -6 -4 -2  0  2  2  1  2
Needleman-Wunsch Global Alignment:
Sequence 1: GAATT--C
Sequence 2: G-ATTAAC
Alignment Score: 2


### Local Sequence Alignment (Smith-Waterman Algorithm)


In [14]:
def smith_waterman(seq1, seq2, match_score=1, mismatch_score=-1, gap_penalty=-1):
    n = len(seq1)
    m = len(seq2)

    # Initialize the score matrix with 0s for local alignment
    score_matrix = [[0] * (m + 1) for _ in range(n + 1)]

    # To find the maximum score and its position for traceback
    max_score = 0
    max_i = 0
    max_j = 0

    # Fill the score matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match = score_matrix[i-1][j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score)
            delete = score_matrix[i-1][j] + gap_penalty
            insert = score_matrix[i][j-1] + gap_penalty

            # For local alignment, score cannot be negative. If all options are negative, set to 0.
            score_matrix[i][j] = max(0, match, delete, insert)

            if score_matrix[i][j] > max_score:
                max_score = score_matrix[i][j]
                max_i = i
                max_j = j

    # Print the score matrix with sequence labels
    print("\nSmith-Waterman Score Matrix:")
    header_elements = ["   "] + [" _ "] + [f"{char:3s}" for char in seq2]
    print("".join(header_elements))

    for i in range(n + 1):
        row_prefix = [" _ "] if i == 0 else [f"{seq1[i-1]:3s}"]
        row_values = [f"{val:3d}" for val in score_matrix[i]]
        print("".join(row_prefix + row_values))

    # Traceback to reconstruct the local alignment
    aligned_seq1 = []
    aligned_seq2 = []
    i, j = max_i, max_j

    while i > 0 and j > 0 and score_matrix[i][j] != 0: # Stop when score becomes 0
        current_score = score_matrix[i][j]
        score_diag = score_matrix[i-1][j-1]
        score_up = score_matrix[i-1][j]
        score_left = score_matrix[i][j-1]

        if current_score == (score_diag + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score)): # Diagonal
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append(seq2[j-1])
            i -= 1
            j -= 1
        elif current_score == (score_up + gap_penalty): # Up
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append('-')
            i -= 1
        else: # Left
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j-1])
            j -= 1

    return ''.join(aligned_seq1[::-1]), ''.join(aligned_seq2[::-1]), max_score

# Example Usage for Smith-Waterman
seq1_sw = "TGTTACGG"
seq2_sw = "GGTTGAC"
aligned1_sw, aligned2_sw, score_sw = smith_waterman(seq1_sw, seq2_sw)

print(f"\nSmith-Waterman Local Alignment:")
print(f"Sequence 1: {aligned1_sw}")
print(f"Sequence 2: {aligned2_sw}")
print(f"Alignment Score: {score_sw}")


Smith-Waterman Score Matrix:
    _ G  G  T  T  G  A  C  
 _   0  0  0  0  0  0  0  0
T    0  0  0  1  1  0  0  0
G    0  1  1  0  0  2  1  0
T    0  0  0  2  1  1  1  0
T    0  0  0  1  3  2  1  0
A    0  0  0  0  2  2  3  2
C    0  0  0  0  1  1  2  4
G    0  1  1  0  0  2  1  3
G    0  1  2  1  0  1  1  2

Smith-Waterman Local Alignment:
Sequence 1: GTT-AC
Sequence 2: GTTGAC
Alignment Score: 4
